# Effects Processing with audio-dsp

This notebook demonstrates the audio effects available in `audio-dsp`:
- Distortion & Saturation
- Filters
- Modulation (Phaser, Chorus)
- Dynamics (Compression)
- Spectral Effects (Vocoder)
- Lo-Fi & Character Effects

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Metallicode/python_audio_dsp/blob/master/examples/Effects.ipynb)

In [ ]:
!pip install audio-dsp -q

In [ ]:
## Load example .wav files to runtime (for Google Colab)
import os
import requests

# Define where to save the files in Colab
output_dir = "/content/assets"
os.makedirs(output_dir, exist_ok=True)

# Define the API URL for the assets folder
api_url = "https://api.github.com/repos/Metallicode/python_audio_dsp/contents/examples/assets?ref=master"

try:
    # Get the list of files from GitHub
    response = requests.get(api_url)
    
    if response.status_code == 200:
        files = response.json()
        print(f"Found {len(files)} files in 'examples/assets'...")
        
        # Download each file
        for file in files:
            if file['type'] == 'file':
                print(f"   Downloading {file['name']}...")
                raw_url = file['download_url']
                file_content = requests.get(raw_url).content
                
                with open(os.path.join(output_dir, file['name']), 'wb') as f:
                    f.write(file_content)
        
        print("\nSuccess! Files are ready in:", output_dir)
        print("Files:", os.listdir(output_dir))
        
    elif response.status_code == 404:
        print("Error 404: The folder 'examples/assets' was not found.")
    else:
        print(f"GitHub API Error: {response.status_code}")
        
except Exception as e:
    print(f"Download failed: {e}")
    print("Will use synthesized sounds instead.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

from audio_dsp.utils import load_audio, save_audio, normalize_audio
from audio_dsp.synth import SubtractiveSynth

SAMPLE_RATE = 44100

# Visualization helpers
def plot_waveform(audio, sr, title="Waveform", figsize=(12, 3), color='#2563eb'):
    """Plot a waveform with time axis."""
    plt.figure(figsize=figsize)
    time = np.arange(len(audio)) / sr
    plt.plot(time, audio, color=color, linewidth=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(title)
    plt.xlim(0, time[-1])
    plt.ylim(-1.1, 1.1)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_comparison(audio1, audio2, sr, title1="Original", title2="Processed", figsize=(12, 5)):
    """Plot two waveforms for comparison."""
    fig, axes = plt.subplots(2, 1, figsize=figsize, sharex=True)
    time1 = np.arange(len(audio1)) / sr
    time2 = np.arange(len(audio2)) / sr
    
    axes[0].plot(time1, audio1, color='#2563eb', linewidth=0.5)
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(title1)
    axes[0].set_ylim(-1.1, 1.1)
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(time2, audio2, color='#dc2626', linewidth=0.5)
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude')
    axes[1].set_title(title2)
    axes[1].set_ylim(-1.1, 1.1)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_spectrum(audio, sr, title="Frequency Spectrum", figsize=(12, 3)):
    """Plot the frequency spectrum."""
    n = len(audio)
    fft = np.abs(np.fft.rfft(audio))
    freqs = np.fft.rfftfreq(n, 1/sr)
    fft_db = 20 * np.log10(fft + 1e-10)
    
    plt.figure(figsize=figsize)
    plt.plot(freqs, fft_db, color='#2563eb', linewidth=0.5)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.title(title)
    plt.xlim(20, 15000)
    plt.xscale('log')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Test Sounds

We'll try to load audio files first. If not available, we'll create synthesized test sounds.

In [ ]:
# Try to load audio files first, fall back to synthesis
import os

# Check for Colab or local paths
if os.path.exists("/content/assets"):
    assets_dir = "/content/assets"
elif os.path.exists("assets"):
    assets_dir = "assets"
else:
    assets_dir = None

loaded_files = False

if assets_dir:
    try:
        # load_audio returns (sr, audio) - must unpack!
        guitar_path = os.path.join(assets_dir, "guitar.wav")
        drums_path = os.path.join(assets_dir, "drums.wav")
        synth_path = os.path.join(assets_dir, "synth.wav")
        
        if os.path.exists(guitar_path):
            _, guitar = load_audio(guitar_path)
            print(f"Loaded guitar: {len(guitar)/SAMPLE_RATE:.2f}s")
            loaded_files = True
        
        if os.path.exists(drums_path):
            _, drums = load_audio(drums_path)
            print(f"Loaded drums: {len(drums)/SAMPLE_RATE:.2f}s")
        
        if os.path.exists(synth_path):
            _, synth_pad = load_audio(synth_path)
            print(f"Loaded synth: {len(synth_pad)/SAMPLE_RATE:.2f}s")
            
    except Exception as e:
        print(f"Error loading files: {e}")
        loaded_files = False

# Fall back to synthesized sounds if files not loaded
if not loaded_files:
    print("Creating synthesized test sounds...")
    
    synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)
    
    # Guitar-like sound
    synth.osc_wave = "saw"
    synth.filter_cutoff = 2000
    synth.filter_resonance = 1.5
    synth.attack = 0.01
    synth.decay = 0.3
    synth.sustain = 0.5
    synth.release = 0.3
    
    guitar = np.concatenate([
        synth.synthesize(110, 0.5),
        synth.synthesize(146.8, 0.5),
        synth.synthesize(110, 0.5),
        synth.synthesize(164.8, 0.5)
    ])
    
    # Drum-like sound
    from audio_dsp.synth.chip_tone import generate_kick, generate_snare, generate_cymbal
    
    step_len = int(SAMPLE_RATE * 0.125)
    def pad_sound(sound, length):
        if len(sound) >= length:
            return sound[:length]
        return np.pad(sound, (0, length - len(sound)))
    
    drums = []
    for i in range(16):
        step = np.zeros(step_len)
        if i in [0, 8]: step += pad_sound(generate_kick(), step_len)
        if i in [4, 12]: step += pad_sound(generate_snare(), step_len)
        if i % 2 == 0: step += pad_sound(generate_cymbal(), step_len) * 0.3
        drums.append(step)
    drums = np.clip(np.concatenate(drums), -1, 1)
    
    # Synth pad
    synth.osc_wave = "triangle"
    synth.filter_cutoff = 1500
    synth.attack = 0.2
    synth.release = 0.5
    synth_pad = synth.synthesize(220, 2.0)
    
    print(f"  Guitar: {len(guitar)/SAMPLE_RATE:.2f}s")
    print(f"  Drums: {len(drums)/SAMPLE_RATE:.2f}s")
    print(f"  Synth: {len(synth_pad)/SAMPLE_RATE:.2f}s")

print("\nTest sounds ready!")

## 1. Distortion & Saturation

19 different distortion types from subtle warmth to extreme destruction.

In [ ]:
from audio_dsp.effects import (
    fuzz_distortion,
    overdrive_distortion,
    saturation_distortion,
    wavefold_distortion,
    bitcrush_distortion
)

print("Original:")
plot_waveform(guitar, SAMPLE_RATE, "Original")
display(Audio(guitar, rate=SAMPLE_RATE))

# Subtle saturation
saturated = saturation_distortion(guitar, gain=1.5, threshold=0.7, mix=0.7)
print("\nSaturation (subtle warmth):")
plot_comparison(guitar, saturated, SAMPLE_RATE, "Original", "Saturation")
display(Audio(saturated, rate=SAMPLE_RATE))

# Overdrive
overdriven = overdrive_distortion(guitar, gain=3.0, threshold=0.5, mix=0.8)
print("\nOverdrive (tube-style):")
plot_comparison(guitar, overdriven, SAMPLE_RATE, "Original", "Overdrive")
display(Audio(overdriven, rate=SAMPLE_RATE))

# Heavy fuzz
fuzzed = fuzz_distortion(guitar, gain=12.0, threshold=0.2, mix=0.9)
print("\nFuzz (extreme):")
plot_comparison(guitar, fuzzed, SAMPLE_RATE, "Original", "Fuzz")
display(Audio(fuzzed, rate=SAMPLE_RATE))

In [ ]:
# Wavefold distortion - great for synths
print("Original synth:")
display(Audio(synth_pad, rate=SAMPLE_RATE))

folded = wavefold_distortion(synth_pad, gain=5.0, mix=0.8)
print("\nWavefold (harmonic richness):")
plot_comparison(synth_pad, folded, SAMPLE_RATE, "Original", "Wavefold")
display(Audio(folded, rate=SAMPLE_RATE))

# Bitcrush for lo-fi
crushed = bitcrush_distortion(synth_pad, bits=6, mix=1.0)
print("\nBitcrush (6-bit):")
plot_comparison(synth_pad, crushed, SAMPLE_RATE, "Original", "Bitcrush")
display(Audio(crushed, rate=SAMPLE_RATE))

## 2. Filters

Resonant ladder-style filters with analog character.

In [ ]:
from audio_dsp.effects import filter_effect

print("Original drums:")
display(Audio(drums, rate=SAMPLE_RATE))

# Low-pass filter
lp_filtered = filter_effect(drums, SAMPLE_RATE, cutoff=500, resonance=2.0, filter_type="lowpass")
print("\nLow-pass (500 Hz, resonance=2):")
plot_comparison(drums, lp_filtered, SAMPLE_RATE, "Original", "Low-pass 500Hz")
display(Audio(lp_filtered, rate=SAMPLE_RATE))

# High resonance for acid-style sound
acid = filter_effect(drums, SAMPLE_RATE, cutoff=800, resonance=4.5, filter_type="lowpass")
print("\nAcid-style (800 Hz, high resonance):")
plot_comparison(drums, acid, SAMPLE_RATE, "Original", "Acid Filter")
display(Audio(acid, rate=SAMPLE_RATE))

In [ ]:
# Filter sweep visualization
# Create a rich synth to filter
synth.osc_wave = "saw"
synth.filter_cutoff = 8000
sweep_source = synth.synthesize(110, 3.0)

# Process in chunks with changing cutoff
chunk_size = int(SAMPLE_RATE * 0.03)
n_chunks = len(sweep_source) // chunk_size

swept = []
cutoffs = []
for i in range(n_chunks):
    chunk = sweep_source[i*chunk_size:(i+1)*chunk_size]
    # Sweep cutoff from 200 to 4000 Hz and back
    progress = i / n_chunks
    cutoff = 200 + 3800 * np.sin(progress * np.pi)
    cutoffs.append(cutoff)
    filtered_chunk = filter_effect(chunk, SAMPLE_RATE, cutoff=cutoff, resonance=3.0)
    swept.append(filtered_chunk)

swept = np.concatenate(swept)

# Plot the sweep
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

time = np.arange(len(swept)) / SAMPLE_RATE
axes[0].plot(time, swept, color='#2563eb', linewidth=0.3)
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Filter Sweep Waveform')
axes[0].set_ylim(-1.1, 1.1)
axes[0].grid(True, alpha=0.3)

cutoff_time = np.linspace(0, len(swept)/SAMPLE_RATE, len(cutoffs))
axes[1].plot(cutoff_time, cutoffs, color='#dc2626', linewidth=2)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Cutoff (Hz)')
axes[1].set_title('Filter Cutoff Automation')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Filter sweep (200 Hz -> 4000 Hz -> 200 Hz):")
Audio(swept, rate=SAMPLE_RATE)

## 3. Vocoder

Classic vocoder effect for robotic voice sounds.

In [ ]:
from audio_dsp.effects.vocoder import vocoder
from audio_dsp.utils import white_noise

# Create a modulator (speech-like: amplitude-modulated noise)
duration = 2.0
t = np.linspace(0, duration, int(duration * SAMPLE_RATE))

# Simulate speech-like modulation
modulation = np.abs(np.sin(2 * np.pi * 3 * t)) * np.abs(np.sin(2 * np.pi * 0.5 * t))
noise_mod = white_noise(len(t)) * modulation * 0.5
noise_mod = filter_effect(noise_mod, SAMPLE_RATE, cutoff=3000, resonance=1.0)

# Create carrier (synth pad)
synth.osc_wave = "saw"
synth.filter_cutoff = 5000
carrier = synth.synthesize(110, duration)

print("Carrier (synth):")
display(Audio(carrier, rate=SAMPLE_RATE))

print("\nModulator (noise):")
display(Audio(noise_mod, rate=SAMPLE_RATE))

# Apply vocoder
vocoded, _ = vocoder(
    carrier=carrier,
    modulator=noise_mod,
    sr=SAMPLE_RATE,
    n_filters=32,
    freq_range=(80, 8000)
)

print("\nVocoded output:")
plot_comparison(carrier, vocoded, SAMPLE_RATE, "Carrier", "Vocoded")
Audio(vocoded, rate=SAMPLE_RATE)

In [ ]:
# Robot voice with generated sawtooth carrier
robot_voice, _ = vocoder(
    carrier=None,  # Generate carrier
    modulator=noise_mod,
    sr=SAMPLE_RATE,
    carrier_type="sawtooth",
    carrier_freq=110,
    n_filters=48
)

plot_waveform(robot_voice, SAMPLE_RATE, "Robot Voice (Generated Sawtooth Carrier)")

print("Robot voice (sawtooth carrier @ 110 Hz):")
Audio(robot_voice, rate=SAMPLE_RATE)

## 4. Modulation Effects

Pitch modulation effects for movement and character.

In [ ]:
from audio_dsp.effects import flutter_effect, pitch_drift, tape_saturation

# Pitch flutter (vibrato/wow effect)
fluttered = flutter_effect(
    guitar,
    sample_rate=SAMPLE_RATE,
    base_rate=5.0,
    rate_diff=1.0,
    depth=0.02
)
print("Pitch flutter (vibrato):")
plot_comparison(guitar, fluttered, SAMPLE_RATE, "Original", "Flutter")
display(Audio(fluttered, rate=SAMPLE_RATE))

# Pitch drift (slow random detuning)
drifted = pitch_drift(
    guitar,
    sample_rate=SAMPLE_RATE,
    drift_depth=0.3,
    drift_rate=0.05
)
print("\nPitch drift (tape-like):")
plot_comparison(guitar, drifted, SAMPLE_RATE, "Original", "Drift")
display(Audio(drifted, rate=SAMPLE_RATE))

In [ ]:
# Tape saturation
taped = tape_saturation(
    guitar,
    sample_rate=SAMPLE_RATE,
    drive=2.0,
    warmth=0.7,
    output_level=1.0
)

print("Tape saturation:")
plot_comparison(guitar, taped, SAMPLE_RATE, "Original", "Tape Saturation")
Audio(taped, rate=SAMPLE_RATE)

## 5. Multi-Band Processing

Split audio into frequency bands and process independently.

In [ ]:
from audio_dsp.effects.multi_band_saturation import process_multi_band

# 3-band saturation: warm lows, crunchy mids, clean highs
multiband = process_multi_band(
    drums,
    fs=SAMPLE_RATE,
    crossovers=[200, 2000],
    drives=[2.0, 4.0, 1.0]
)

print("Multi-band saturation (warm lows, crunchy mids, clean highs):")
plot_comparison(drums, multiband, SAMPLE_RATE, "Original Drums", "Multi-band Saturated")

# Show spectrum comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

for i, (audio, title) in enumerate([(drums, "Original"), (multiband, "Multi-band")]):
    fft = np.abs(np.fft.rfft(audio))
    freqs = np.fft.rfftfreq(len(audio), 1/SAMPLE_RATE)
    fft_db = 20 * np.log10(fft + 1e-10)
    axes[i].plot(freqs, fft_db, linewidth=0.5)
    axes[i].set_xlabel('Frequency (Hz)')
    axes[i].set_ylabel('Magnitude (dB)')
    axes[i].set_title(f'{title} Spectrum')
    axes[i].set_xlim(20, 15000)
    axes[i].set_xscale('log')
    axes[i].axvline(200, color='r', linestyle='--', alpha=0.5, label='Crossover')
    axes[i].axvline(2000, color='r', linestyle='--', alpha=0.5)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

display(Audio(drums, rate=SAMPLE_RATE))
print("\nMulti-band:")
display(Audio(multiband, rate=SAMPLE_RATE))

## 6. Effect Chains

Combine multiple effects for complex sound design.

In [ ]:
# Guitar pedalboard simulation
signal = guitar.copy()
chain_stages = [('Original', signal.copy())]

# 1. Overdrive
signal = overdrive_distortion(signal, gain=2.5, threshold=0.6, mix=0.7)
chain_stages.append(('+ Overdrive', signal.copy()))

# 2. Filter (tone control)
signal = filter_effect(signal, SAMPLE_RATE, cutoff=3000, resonance=1.0)
chain_stages.append(('+ Filter', signal.copy()))

# 3. Tape saturation for warmth
signal = tape_saturation(signal, sample_rate=SAMPLE_RATE, drive=1.5, warmth=0.5, output_level=1.0)
chain_stages.append(('+ Tape', signal.copy()))

# 4. Normalize
signal = normalize_audio(signal, peak=0.9)

# Visualize the chain
fig, axes = plt.subplots(len(chain_stages), 1, figsize=(12, 10))

for i, (name, audio) in enumerate(chain_stages):
    time = np.arange(len(audio)) / SAMPLE_RATE
    axes[i].plot(time, audio, color='#2563eb', linewidth=0.5)
    axes[i].set_ylabel(name)
    axes[i].set_ylim(-1.1, 1.1)
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (s)')
plt.suptitle('Effect Chain Visualization')
plt.tight_layout()
plt.show()

print("Original:")
display(Audio(guitar, rate=SAMPLE_RATE))

print("\nFinal (Overdrive -> Filter -> Tape):")
display(Audio(signal, rate=SAMPLE_RATE))

In [ ]:
# Experimental synth processing
signal = synth_pad.copy()

# 1. Wavefold for harmonics
signal = wavefold_distortion(signal, gain=3.0, mix=0.6)

# 2. Bitcrush for digital grit
signal = bitcrush_distortion(signal, bits=10, mix=0.3)

# 3. Resonant filter
signal = filter_effect(signal, SAMPLE_RATE, cutoff=1500, resonance=3.5)

# 4. Pitch flutter for movement
signal = flutter_effect(signal, sample_rate=SAMPLE_RATE, base_rate=3.0, depth=0.01)

signal = normalize_audio(signal, peak=0.9)

plot_comparison(synth_pad, signal, SAMPLE_RATE, "Original Synth", "Experimental Chain")

print("Original synth:")
display(Audio(synth_pad, rate=SAMPLE_RATE))

print("\nExperimental (Wavefold -> Bitcrush -> Filter -> Flutter):")
display(Audio(signal, rate=SAMPLE_RATE))

## 7. Saving Processed Audio

In [ ]:
save_audio("output_effects.wav", signal, SAMPLE_RATE)
print("Saved to output_effects.wav")